In [3]:
!pip install faiss-cpu

  Using cached faiss_cpu-1.8.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (3.6 kB)
Using cached faiss_cpu-1.8.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (27.0 MB)


In [5]:
# for reading scrapped insagram posts
import json
# for finding the closest insagram post to the prompt
from sklearn.feature_extraction.text import TfidfVectorizer
import faiss

In [6]:
# scrapped instagram data
posts_json_path = r'/kaggle/input/instagram-posts/dataset_instagram-scraper_2024-05-14_13-22-46-908.json'

with open(posts_json_path, "r", encoding="utf-8") as file:
    data = json.load(file)

# extract caption i.e main text from post
texts = [item['caption'] for item in data if 'caption' in item]

# vectorize instagram data
tf = TfidfVectorizer()
vectorizer = tf.fit_transform(texts)
vectors = vectorizer.toarray()

d = vectors.shape[1]
index = faiss.IndexFlatL2(d)
index.add(vectors)

def get_relevant_texts(query, top_k=3):
    query_vec = tf.transform([query]).toarray()
    # find the "closest" post
    distances, indices = index.search(query_vec, top_k)
    relevant_texts = [texts[i] for i in indices[0]]
    return relevant_texts

In [7]:
# finding the closest post
query = "new backpack model Adventure for mountains hike"
relevant_texts = get_relevant_texts(query)
print(relevant_texts[1])

Sojourn Porter™ Travel Pack: Part backpack. Part duffel. All convenience. 

#OspreyPacks


In [8]:
# set default device to GPU to accelerate inference
import torch
torch.set_default_device('cuda')

In [9]:
# get the model and toketizer
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "microsoft/phi-2"
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

config.json:   0%|          | 0.00/735 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/35.7k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/564M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [10]:
def generate_post_with_context(main_topic: str, model_name: str, features: str, find_similar: bool =True):
    """
    Main funtion for generating post
    Paremeters:
    main - main topic of the post (backpack, sport etc.)
    model_name - specific model of the equipment (if we need to mention it in the post)
    features - some features of the main topic (for backpack, for example: comfort, 200 liter, 50$, camping etc.)
    find_similar - whether we want to find the closes instagram post and use it for style context
    """
    relevant_texts = ' '
    if find_similar: 
        relevant_texts = get_relevant_texts(features)[0]
#         context = " ".join(relevant_texts)
    if model_name:
        prompt = f"Instruct: Imagine you are a online company selling sport backpacks. Write a short instagram specific post about {main_topic}. Mention only {features} and name of the product {model_name}. Promote activities associated with this product and stick to this style: {relevant_texts} \nOutput:"
    else:
        prompt = f"Instruct: Imagine you are a online selling company. Write a short instagram general post promoting {main_topic}. Write about {features} and stick to this style: {relevant_texts}.\nOutput:"

    inputs = tokenizer.encode(prompt, return_tensors="pt")
    outputs = model.generate(inputs, max_new_tokens=150, num_return_sequences=1)
    result = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return result.split('\nOutput:')[-1].strip()

In [12]:
main = "backpack"
model_name = 'ADV5'
features = 'camping, hiking, 200 liters'
generated_post = generate_post_with_context(main, model_name, features, find_similar=True)
print(generated_post)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


🌲🏕️🥾 Calling all adventure seekers! 🌲🏕️🥾 Introducing the ADV5, the ultimate camping and hiking backpack with a whopping 200 liters of storage space. 🌲🏕️🥾 Whether you're exploring the great outdoors or embarking on a multi-day trek, the ADV5 has got you covered. 🌲🏕️🥾 With its durable construction and comfortable straps, you'll be ready for any adventure that comes your way. 🌲🏕️🥾 Don't miss out on this must-have gear for your next outdoor escapade. 🌲🏕️🥾


In [12]:
!pip install gradio

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 83.1 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.8/315.8 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 89.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.2/47.2 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.1/121.1 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 7.9 MB/s eta 0:00:00
  Created wheel for ffmpy: filename=ffmpy-0.3.2-py3-none-any.whl size=5584 sha256=59c269962305829e860e6c0026743fd4aa10a62de8d0028bd3b3957ae4c1d2da
  Stored in directory: /root/.cache/pip/wheels/bd/65/9a/671fc6dcde07d4418df0c592f8df512b26d7a0029c2a23dd81
Successfully built ffmpy
  Attempting uninstall: websockets
    Found existing installation: websockets 12.0
    Uninstalling websockets-12.0:
      Successfully uninstalled websockets-12.0
  Attempting uninst

In [13]:
import gradio as gr

In [14]:
def gradio_interface(main, model_name, features, find_similar):
    return generate_post_with_context(main, model_name, features, find_similar)

interface = gr.Interface(
    fn=gradio_interface,
    inputs=[
        gr.Textbox(label="Main Topic"),
        gr.Textbox(label="Model Name"),
        gr.Textbox(label="Features"),
        gr.Checkbox(label="Find Similar Texts")
    ],
    outputs="text",
    title="Backpack Promotion Post Generator",
    description="Generate Instagram posts promoting backpacks with specific features."
)

In [15]:
interface.launch()

Running on local URL:  http://127.0.0.1:7860
Kaggle notebooks require sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Running on public URL: https://b1b2d5f6060b42349d.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
